## **1. Descargar votos de MVP con requests**

Se extrae el contenedor que contiene los mejores jugadores por temporada

![MVP1](images/mvp_01.png)

In [1]:
years = list(range(1991, 2022))

In [2]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [3]:
import requests
for year in years:
    url = url_start.format(year)
    data = requests.get(url)

    # w+ : Un archivo para escritura y lectura
    with open("mvp/{}.html".format(year), "w+", encoding= "utf-8") as f:
        f.write(data.text)

## **2. Analizando la tabla de votos con beautifulsoup**

In [4]:
from bs4 import BeautifulSoup

In [5]:
with open("mvp/1991.html", encoding='UTF-8') as f:
    page = f.read()

De manera práctica, eliminaremos el 1er encabezado, para poder solo incluir las filas que nos importa y convertirlo aun DataFrame

![MVP2](images/mvp_02.png)

In [6]:
soup = BeautifulSoup(page, "html.parser")
soup.find('tr', class_ = 'over_header').decompose() #* Elimina el 1er encabezado 
mvp_table = soup.find(id="mvp")

In [7]:
import pandas as pd

In [8]:
mvp_1991 = pd.read_html(str(mvp_table))

In [ ]:
#* Nos devuelve una lista de DataFrames 
mvp_1991 

In [ ]:
mvp_1991 = pd.read_html(str(mvp_table))[0]
mvp_1991

Sacando todos los DataFrames de todas las temporadas en una sola lista

In [11]:
dfs = []
for year in years:
    with open('mvp/{}.html'.format(year), encoding='utf-8') as f:
        page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        soup.find('tr', class_ = 'over_header').decompose()
        mvp_table = soup.find(id='mvp')
        mvp = pd.read_html(str(mvp_table))[0]
        mvp["year"] = year

        dfs.append(mvp)

In [ ]:
dfs[0]

## **3. Combinando votos MVP con pandas**

In [ ]:
mvps = pd.concat(dfs)
mvps.head()

In [47]:
mvps.to_csv('mvps.csv',index=False)

### **3.1. Descarga de estadísticas de jugadores**

Ya que conocemos los MVP de las temporadas 1991-2021, se procederá a captar todos los jugadores de las temporadas, de tal manera poder entrenar un algoritmo de ML que pueda predecir que jugador de la siguietne temporada será el MVP

![PLAYER1](images/player_01.png)

> Al tener el enlace de las estadísticas por jugador/juego, solo me mostrará las primeras 17 filas, y luego mostrará las siguientes 17 filas conforme se vaya haciendo scroll mediante código javascript.

## **4. Usando Selenium para screappear una página de Javascript**

Esto es un problema muy común en el webscrapping, muchos sitios web tendrán javascript u otros componenes

In [14]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [15]:
url = player_stats_url.format(1991)
data = requests.get(url)
with open("player/1991.html", "w+", encoding='utf-8') as f:
    f.write(data.text)

![PLAYER2](images/player_02.png)

Previamente instalamos [ChromeDriver](https://chromedriver.chromium.org/) según tu SO, es un servidor independiente que implementa el protocolo de cable de WebDriver para Chromium. 

In [16]:
from selenium import webdriver

In [17]:
# Crea una ventana que está siendo controlada por Selenium
# Usaremos ese navegador para mostrar la pagina con todas las filas que necesitamos
driver = webdriver.Chrome(executable_path = "C:\ChromeDriver\chromedriver")

C:\Users\sandr\AppData\Local\Temp\ipykernel_28208\4140206510.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = "C:\ChromeDriver\chromedriver")


In [18]:
import time

year = 1991
url = player_stats_url.format(year)

#Renderice la URL en el navegador
driver.get(url) 

#Ejecutamos un poco de javascript en el navegador
 #? Comando de JS que solo dice al navegador para desplazarse hacia abajo
driver.execute_script("window.scrollTo(1,10000)")
time.sleep(2)

html = driver.page_source

In [ ]:
html[0:500]

Guardamos todos los datos de jugadoes por todas las temporadas

In [22]:
for year in years:
    url = player_stats_url.format(year)

    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)

    html = driver.page_source
    with open("player/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(html)

### **4.1. Analizando las estadísticas con beautifulsoup**

Debido que al traer todas las filas de todos los jugadores, me mostrará cabeceras intermedias entre registros, para este caso eliminaremos aquellas cabeceras, para tener netamente registros de jugadores, y eso lo hacemos en el siguiente fraagmento:
``` python
list_theads = soup.find_all('tr', class_ = "thead")
for thead in list_theads:
     thead.decompose()
```

![PLAYER3](images/player_03.png)

In [40]:
dfs = []
for year in years: 
    with open("player/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    list_theads = soup.find_all('tr', class_ = "thead")
    for thead in list_theads:
        thead.decompose()
    player_table = soup.find(id="per_game_stats")
    player = pd.read_html(str(player_table))[0]
    player["year"] = year
    dfs.append(player)

In [43]:
players = pd.concat(dfs)

In [ ]:
players

In [45]:
players.to_csv('players.csv')

## **5. Descarga de datos del equipo**

Cuando predecimos quien ganará el MVP, queremos asegurarnos de insertar el registro de su equipo, para que el modelo de Aprendizaje automatico pueda ver ese registro, y usarlo como predictor. Scrapeamos los records del equipo por año, la posiciones de equipos por conferencia, el cuál corresponde a 2 tablas.

![TEAM1](images/team_01.png)

In [46]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [49]:
for year in years:
    url = team_stats_url.format(year)
    data = requests.get(url)
    with open("team/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)


Las tablas que queremos traer se cargan automáticamente, se necesitaría de javascript para poder cargar el resto del contenido mediante scroll

![TEAM2](images/team_02.png)

## **6. Analizando los datos del equipo con beautifulsoup**

Eliminaremos los titulos de las divisiones de conferencias, para solo tener registros acoplados a las conferencias `este y oeste`

In [54]:
dfs = []
for year in years:
    with open("team/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    list_theads = soup.find_all('tr', class_ = 'thead')
    for thead in list_theads:
        thead.decompose()
    team_table_E = soup.find(id="divs_standings_E")
    team_table_W = soup.find(id="divs_standings_W")

    team_E = pd.read_html(str(team_table_E))[0]
    team_E["year"] = year
    team_E["team"] = team_E['Eastern Conference']
    del team_E['Eastern Conference']
    
    team_W = pd.read_html(str(team_table_W))[0]
    team_W["year"] = year
    team_W["team"] = team_W['Western Conference']
    del team_W['Western Conference']

    dfs.extend([team_E, team_W])



In [ ]:
dfs[0]

### **6.1. Combinando estadísticas de equipo con pandas**

In [ ]:
teams = pd.concat(dfs)
teams

In [60]:
teams.to_csv('teams.csv', index = False)